In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [3]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [18]:
events['operating_system_name'] = events['operating_system_version'].str.split(' ').str[0]
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='Phone', events['operating_system_name'] + ' Phone', events['operating_system_name'])
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='OS', events['operating_system_name'] + ' OS', events['operating_system_name'])

In [19]:
events['browser_name'] = events['browser_version'].str.split(' ').str[0]
events['browser_name'] = np.where(events['browser_version'].str.split(' ').str[1]=='Mobile', events['browser_name'] + ' Mobile', events['browser_name'])

In [20]:
events['marca'] = pd.Categorical(events['model'].str.split(' ').str[0])
events['modelo'] = events['model'].str.split(' ').str[1:].str.join(' ')
events[['marca', 'modelo']].head()

,marca,modelo
0,Samsung,Galaxy J7 Prime
1,iPhone,5s
2,iPhone,6S
3,iPhone,7
4,LG,G4 H818P


In [21]:
#Marcas Vistas
viewed_data = events[['person', 'marca', 'timestamp']].pivot_table(index='person', columns='marca', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
#viewed_data.columns = ['viewed_' + col if col != 'person' else col for col in viewed_data.columns]
viewed_data = viewed_data.drop(viewed_data.columns[1], axis=1)
viewed_data.head()

marca,person,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,0,1,0,0,0,1,0,0,0,1
1,00091926,0,2,0,1,0,61,1,0,1,253
2,00091a7a,0,0,0,0,0,0,0,0,0,3
3,000ba417,0,4,0,0,0,110,0,0,1,8
4,000c79fe,0,0,0,0,0,0,0,0,0,4


In [22]:
## Condiciones de los telefonos
condiciones = events[['person', 'condition', 'timestamp']].pivot_table(index='person', columns='condition', values='timestamp', aggfunc='count', fill_value=0)
condiciones.columns = condiciones.columns.astype('object')
condiciones.reset_index(inplace=True)
#condiciones.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]
viewed_data = pd.merge(condiciones, viewed_data, on='person', how='left').fillna(0)
viewed_data = viewed_data.drop(viewed_data.columns[1], axis=1)
viewed_data.head()

,person,Bom,Bom - Sem Touch ID,Excelente,Muito Bom,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,2,0,0,1,0,1,0,0,0,1,0,0,0,1
1,00091926,102,1,108,163,0,2,0,1,0,61,1,0,1,253
2,00091a7a,1,0,2,0,0,0,0,0,0,0,0,0,0,3
3,000ba417,115,0,11,34,0,4,0,0,0,110,0,0,1,8
4,000c79fe,4,0,0,0,0,0,0,0,0,0,0,0,0,4


In [23]:
## Colores de los telefonos
colores = events[['person', 'color', 'timestamp']].pivot_table(index='person', columns='color', values='timestamp', aggfunc='count', fill_value=0)
colores.columns = colores.columns.astype('object')
colores.reset_index(inplace=True)
#colores.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

viewed_data = pd.merge(colores, viewed_data, on='person', how='left').fillna(0)
viewed_data = viewed_data.drop(viewed_data.columns[1], axis=1)
viewed_data.head()

,person,Amarelo,Ametista,Azul,Azul Escuro,Azul Safira,Azul Topázio,Bambu,Black Piano,Branco,...,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,00091926,0,3,0,0,1,0,29,1,3,...,0,2,0,1,0,61,1,0,1,253
2,00091a7a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,000ba417,1,7,1,0,0,1,0,0,15,...,0,4,0,0,0,110,0,0,1,8
4,000c79fe,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,4


In [24]:
# Capacidades de los telefonos
capacidades = events[['person', 'storage', 'timestamp']].pivot_table(index='person', columns='storage', values='timestamp', aggfunc='count', fill_value=0)
capacidades.columns = capacidades.columns.astype('object')
capacidades.reset_index(inplace=True)
#capacidades.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

viewed_data = pd.merge(capacidades, viewed_data, on='person', how='left').fillna(0)
viewed_data = viewed_data.drop(viewed_data.columns[1], axis=1)
viewed_data.head()
# capacidades

,person,128GB,16GB,256GB,32GB,4GB,64GB,8GB,Amarelo,Ametista,...,Asus,LG,Lenovo,Motorola,Quantum,Samsung,Sony,Xiaomi,iPad,iPhone
0,0008ed71,0,0,0,2,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,00091926,48,0,0,132,0,80,10,0,3,...,0,2,0,1,0,61,1,0,1,253
2,00091a7a,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,000ba417,0,22,1,20,0,1,1,1,7,...,0,4,0,0,0,110,0,0,1,8
4,000c79fe,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [25]:
viewed_data.shape

(38250, 84)

In [26]:
viewed_data.to_csv('features_caracteristicas.csv', index = False)